In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames[:1]:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/renewdata/sample_submission.csv
/kaggle/input/renewdata/test/641.png
/kaggle/input/renewdata/train/가구수정/11.png
/kaggle/input/renewdata/train/터짐/94.png
/kaggle/input/renewdata/train/들뜸/48.png
/kaggle/input/renewdata/train/피스/48.png
/kaggle/input/renewdata/train/창틀,문틀수정/11.png
/kaggle/input/renewdata/train/녹오염/11.png
/kaggle/input/renewdata/train/석고수정/48.png
/kaggle/input/renewdata/train/훼손/1231.png
/kaggle/input/renewdata/train/꼬임/173.png
/kaggle/input/renewdata/train/울음/11.png
/kaggle/input/renewdata/train/걸레받이수정/173.png
/kaggle/input/renewdata/train/틈새과다/4.png
/kaggle/input/renewdata/train/오염/173.png
/kaggle/input/renewdata/train/곰팡이/94.png
/kaggle/input/renewdata/train/이음부불량/11.png
/kaggle/input/renewdata/train/면불량/94.png
/kaggle/input/renewdata/train/반점/1.png
/kaggle/input/renewdata/train/오타공/94.png
/kaggle/input/renewdata/train/몰딩수정/94.png
/kaggle/input/csvfile/ISSUE_HW_DAY_2021-07_2021-07_2021.csv


augmented images 생성해둔 것

In [2]:
import random
import pandas as pd
import numpy as np
import os
import re
import glob
import cv2

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, WeightedRandomSampler

import albumentations as A
from albumentations.pytorch.transforms import ToTensorV2
import torchvision.models as models

from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.metrics import f1_score
from sklearn.metrics import classification_report
from tqdm.auto import tqdm

import warnings
warnings.filterwarnings(action='ignore') 

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [3]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')


CFG = {
    'IMG_SIZE':384,
    'EPOCHS':50,
    'LEARNING_RATE':2e-4,
    'BATCH_SIZE':32,
    'SEED':42
}


def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

seed_everything(CFG['SEED']) # Seed...

augmented images 코드


In [5]:
all_img_list = glob.glob('/kaggle/input/renewdata/train/*/*')

df = pd.DataFrame(columns=['img_path', 'label'])
df['img_path'] = all_img_list
df['label'] = df['img_path'].apply(lambda x : str(x).split('/')[-2])
df['label'].value_counts()

훼손         1405
오염          595
걸레받이수정      307
꼬임          210
터짐          162
곰팡이         145
오타공         142
몰딩수정        130
면불량          99
석고수정         57
들뜸           54
피스           51
창틀,문틀수정      27
울음           22
이음부불량        17
녹오염          14
가구수정         12
틈새과다          5
반점            3
Name: label, dtype: int64

In [7]:
le = preprocessing.LabelEncoder()
df['label'] = le.fit_transform(df['label'])

In [10]:
class CustomDataset(Dataset):
    def __init__(self, img_path_list, label_list, transforms=None):
        self.img_path_list = img_path_list
        self.label_list = label_list
        self.transforms = transforms
        
    def __getitem__(self, index):
        img_path = self.img_path_list[index]
        
        image = cv2.imread(img_path)
        
        if self.transforms is not None:
            image = self.transforms(image=image)['image']
        
        if self.label_list is not None:
            label = self.label_list[index]
            return image, label
        else:
            return image
        
    def __len__(self):
        return len(self.img_path_list)

In [11]:
train_transform = A.Compose([
                            A.Resize(CFG['IMG_SIZE'],CFG['IMG_SIZE']),
                            A.HorizontalFlip(),
                            A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225), max_pixel_value=255.0, always_apply=False, p=1.0),
                            ToTensorV2()
                            ])

test_transform = A.Compose([
                            A.Resize(CFG['IMG_SIZE'],CFG['IMG_SIZE']),
                            A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225), max_pixel_value=255.0, always_apply=False, p=1.0), # ImageNet에서 쓰는 픽셀 분포
                            ToTensorV2()
                            ])

In [12]:
class BaseModel(nn.Module):
    def __init__(self, num_classes=len(le.classes_)):
        super(BaseModel, self).__init__()
        self.backbone = models.efficientnet_b1(pretrained=True)
        self.classifier = nn.Linear(1000, num_classes)
        
    def forward(self, x):
        x = self.backbone(x)
        x = self.classifier(x)
        return x

In [13]:
def train(model, optimizer, train_loader, val_loader, scheduler, device):
    model.to(device)
    criterion = nn.CrossEntropyLoss().to(device)
    
    best_score = 0
    best_model = None
    num_patience = 7
    cur_patience = 0
    for epoch in range(1, CFG['EPOCHS']+1):
        model.train()
        train_loss = []
        for imgs, labels in tqdm(iter(train_loader)):
            imgs = imgs.float().to(device)
            labels = labels.to(device)
            
            optimizer.zero_grad()
            
            output = model(imgs)
            loss = criterion(output, labels)
            
            loss.backward()
            optimizer.step()
            
            train_loss.append(loss.item())
                    
        _val_loss, _val_score = validation(model, criterion, val_loader, device)
        _train_loss = np.mean(train_loss)
        print(f'Epoch [{epoch}], Train Loss : [{_train_loss:.5f}] Val Loss : [{_val_loss:.5f}] Val Weighted F1 Score : [{_val_score:.5f}]')
       
        if scheduler is not None:
            scheduler.step(_val_score)
            
        if best_score < _val_score:
            best_score = _val_score
            best_model = model
            cur_patience = 0
        else :
            cur_patience+=1
            print(cur_patience)
            if cur_patience>=num_patience:
                break
    return best_model

def validation(model, criterion, val_loader, device):
    model.eval()
    val_loss = []
    

    with torch.no_grad():
        for imgs, labels in tqdm(iter(val_loader)):
            imgs = imgs.float().to(device)
            labels = labels.to(device)
            
            pred = model(imgs)
            
            loss = criterion(pred, labels)
            
            preds += pred.argmax(1).detach().cpu().numpy().tolist()
            true_labels += labels.detach().cpu().numpy().tolist()
            
            val_loss.append(loss.item())
        
        _val_loss = np.mean(val_loss)
        _val_score = f1_score(true_labels, preds, average='weighted')
    
    return _val_loss, _val_score

In [14]:
def inference(model, test_loader, device):
    model.eval()
    preds = []
    with torch.no_grad():
        for imgs in tqdm(iter(test_loader)):
            imgs = imgs.float().to(device)
            
            pred = model(imgs)
            
            preds += pred.argmax(1).detach().cpu().numpy().tolist()
    
    preds = le.inverse_transform(preds)  ## 앙상블 할땐 확률값 반환
    return preds

In [16]:
from sklearn.model_selection import StratifiedKFold
skf = StratifiedKFold(n_splits = 3, shuffle=True, random_state = 45) ## n_splits >= 5

test = pd.read_csv('/kaggle/input/renewdata/test.csv')
test["img_path"] = "/kaggle/input/renewdata" + test["img_path"].str[1:]

result = []
for train_index, valid_index in skf.split(df, df["label"]):
    x_train = df.iloc[train_index]
    x_valid = df.iloc[valid_index]
    train_dataset = CustomDataset(x_train['img_path'].values, x_train['label'].values, train_transform)
    train_loader = DataLoader(train_dataset, batch_size = CFG['BATCH_SIZE'], shuffle=True, num_workers=0)

    val_dataset = CustomDataset(x_valid['img_path'].values, x_valid['label'].values, test_transform)
    val_loader = DataLoader(val_dataset, batch_size=CFG['BATCH_SIZE'], shuffle=False, num_workers=0)
    
    
    model = BaseModel()
    model.eval()
    optimizer = torch.optim.Adam(params = model.parameters(), lr = CFG["LEARNING_RATE"])
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', factor=0.2, patience=3, threshold_mode='abs', min_lr=1e-8, verbose=True)

    infer_model = train(model, optimizer, train_loader, val_loader, scheduler, device)
    

    test_dataset = CustomDataset(test['img_path'].values, None, test_transform)
    test_loader = DataLoader(test_dataset, batch_size=CFG['BATCH_SIZE'], shuffle=False, num_workers=0)
    
    preds = inference(infer_model, test_loader, device)
    result.append(preds)   ## hard/soft voting
    ## TTA 추가 예정


Downloading: "https://download.pytorch.org/models/efficientnet_b1_rwightman-533bc792.pth" to /root/.cache/torch/hub/checkpoints/efficientnet_b1_rwightman-533bc792.pth
100%|██████████| 30.1M/30.1M [00:00<00:00, 114MB/s] 


  0%|          | 0/72 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:
submit = pd.read_csv('/kaggle/input/renewdata/sample_submission.csv')
submit['label'] = preds
submit.to_csv('baseline_submit.csv', index=False)
submit